In [1]:
%pip install pandas numpy scikit-learn gensim lazypredict lightautoml "flaml[blendsearch]" matplotlib seaborn -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
import warnings
import re
import matplotlib.pyplot as plt
import seaborn as sns

from lazypredict.Supervised import LazyClassifier, LazyRegressor
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from flaml import AutoML

warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
print("Библиотеки импортированы.")

'nlp' extra dependency package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependency package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
Библиотеки импортированы.


In [3]:
def preprocess_churn(train_path, test_path, target_column='Churn'):
    """Предобработка данных для задачи Churn."""
    print(f"Preprocessing Churn data: {train_path}, {test_path}...")
    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)

    df_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df_train.columns]
    df_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df_test.columns]
    target_column = "".join (c if c.isalnum() else "_" for c in str(target_column))

    df_train['TotalSpent'] = pd.to_numeric(df_train['TotalSpent'], errors='coerce')
    df_train['TotalSpent'] = df_train['TotalSpent'].fillna(0)

    df_test['TotalSpent'] = pd.to_numeric(df_test['TotalSpent'], errors='coerce')
    df_test['TotalSpent'] = df_test['TotalSpent'].fillna(0)

    X_train = df_train.drop(target_column, axis=1)
    y_train = df_train[target_column]
    X_test = df_test.drop(target_column, axis=1)
    y_test = df_test[target_column]

    categorical_cols_train = X_train.select_dtypes(include=['object', 'category']).columns
    categorical_cols_test = X_test.select_dtypes(include=['object', 'category']).columns
    common_categorical_cols = list(set(categorical_cols_train) & set(categorical_cols_test))

    for col in common_categorical_cols:
        X_train[col] = X_train[col].astype(str)
        X_test[col] = X_test[col].astype(str)

        unique_vals_train = set(X_train[col].unique())
        map_dict = {'Yes': 1, 'No': 0, 'No phone service': 0, 'No internet service': 0}
        if unique_vals_train.issubset(set(map_dict.keys()) | {np.nan, 'nan', 'None'}):
            X_train[col] = X_train[col].map(map_dict).fillna(0).astype(int)
            X_test[col] = X_test[col].map(map_dict).fillna(0).astype(int)
            common_categorical_cols.remove(col)

    if common_categorical_cols:
        X_train = pd.get_dummies(X_train, columns=common_categorical_cols, dummy_na=False, drop_first=False)
        X_test = pd.get_dummies(X_test, columns=common_categorical_cols, dummy_na=False, drop_first=False)

        train_cols = set(X_train.columns)
        test_cols = set(X_test.columns)

        missing_in_test = list(train_cols - test_cols)
        for c in missing_in_test:
            if c != target_column:
                X_test[c] = 0

        missing_in_train = list(test_cols - train_cols)
        for c in missing_in_train:
             if c != target_column:
                X_train[c] = 0

        final_train_cols = [col for col in X_train.columns if col != target_column]
        X_test = X_test[final_train_cols]
    else:
         print("No remaining categorical columns for One-Hot Encoding.")


    print("Churn preprocessing finished.")
    return X_train, y_train, X_test, y_test


def preprocess_price(train_path, test_path, target_column='price'):
    """Предобработка данных для задачи Price."""
    print(f"Preprocessing Price data: {train_path}, {test_path}...")
    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)

    df_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df_train.columns]
    df_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df_test.columns]
    target_column = "".join (c if c.isalnum() else "_" for c in str(target_column))

    X_train = df_train.drop(target_column, axis=1)
    y_train = df_train[target_column]
    X_test = df_test.drop(target_column, axis=1)
    y_test = df_test[target_column]

    numeric_cols_train = X_train.select_dtypes(include=np.number).columns
    numeric_cols_test = X_test.select_dtypes(include=np.number).columns
    common_numeric_cols = list(set(numeric_cols_train) & set(numeric_cols_test))

    for col in common_numeric_cols:
        median_val = X_train[col].median()
        X_train[col].fillna(median_val, inplace=True)
        X_test[col].fillna(median_val, inplace=True)

    categorical_cols_train = X_train.select_dtypes(include=['object', 'category']).columns
    categorical_cols_test = X_test.select_dtypes(include=['object', 'category']).columns
    common_categorical_cols = list(set(categorical_cols_train) & set(categorical_cols_test))

    for col in common_categorical_cols:
        X_train[col] = X_train[col].astype(str)
        X_test[col] = X_test[col].astype(str)

        mode_val = X_train[col].mode()[0] if not X_train[col].mode().empty else "Missing"
        X_train[col].fillna(mode_val, inplace=True)
        X_test[col].fillna(mode_val, inplace=True)

        train_categories = set(X_train[col].unique())
        test_categories = set(X_test[col].unique())
        new_categories_in_test = list(test_categories - train_categories)
        if new_categories_in_test:
             print(f"Warning: New categories found in test column '{col}': {new_categories_in_test}. Replacing with mode '{mode_val}'.")
             X_test[col] = X_test[col].apply(lambda x: mode_val if x in new_categories_in_test else x)

    if common_categorical_cols:
        X_train = pd.get_dummies(X_train, columns=common_categorical_cols, dummy_na=False, drop_first=False) # Используем общее подмножество колонок
        X_test = pd.get_dummies(X_test, columns=common_categorical_cols, dummy_na=False, drop_first=False)

        train_cols = set(X_train.columns)
        test_cols = set(X_test.columns)

        missing_in_test = list(train_cols - test_cols)
        for c in missing_in_test:
             if c != target_column:
                X_test[c] = 0

        missing_in_train = list(test_cols - train_cols)
        for c in missing_in_train:
            if c != target_column:
                X_train[c] = 0

        final_train_cols = [col for col in X_train.columns if col != target_column]
        X_test = X_test[final_train_cols]
    else:
        print("No categorical columns found for One-Hot Encoding.")

    non_numeric_train_cols = X_train.select_dtypes(exclude=np.number).columns
    if len(non_numeric_train_cols) > 0:
        print(f"Warning: Dropping remaining non-numeric columns from train/test: {list(non_numeric_train_cols)}")
        X_train = X_train.drop(columns=non_numeric_train_cols)
        cols_to_drop_in_test = [col for col in non_numeric_train_cols if col in X_test.columns]
        X_test = X_test.drop(columns=cols_to_drop_in_test)

    print("Price preprocessing finished.")
    return X_train, y_train, X_test, y_test


print("Preprocessing functions defined.")

Preprocessing functions defined.


In [4]:
def run_lazypredict(X_train, y_train, X_test, y_test, task_type):
    """Запускает LazyPredict и возвращает метрику и время."""
    print(f"\n--- Running LazyPredict for {task_type} ---")
    start_time = time.time()
    best_model_metric = np.nan
    models = None

    try:
        if task_type == 'classification':
            le = LabelEncoder()
            y_train_enc = le.fit_transform(y_train)
            y_test_enc = le.transform(y_test)

            clf = LazyClassifier(verbose=1, ignore_warnings=True, custom_metric=roc_auc_score, predictions=False)
            models, _ = clf.fit(X_train, X_test, y_train_enc, y_test_enc)
            metric_name = 'ROC AUC'
            if models is not None and not models.empty and metric_name in models.columns:
                best_model_metric = models[metric_name].max()
            else:
                print("LazyClassifier did not return models or the metric column.")
                best_model_metric = 'N/A'

        elif task_type == 'regression':
            reg = LazyRegressor(verbose=1, ignore_warnings=True, custom_metric=lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), predictions=False)
            y_train_clean = y_train.fillna(y_train.median())
            y_test_clean = y_test.fillna(y_train.median())
            models, _ = reg.fit(X_train, X_test, y_train_clean, y_test_clean)
            metric_name = 'RMSE'
            if models is not None and not models.empty and metric_name in models.columns:
                best_model_metric = models[metric_name].min()
            else:
                print("LazyRegressor did not return models or the metric column.")
                best_model_metric = 'N/A'

        else:
            raise ValueError("task_type must be 'classification' or 'regression'")

    except Exception as e:
        print(f"LazyPredict failed: {e}")
        best_model_metric = 'Error'

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"LazyPredict finished in {elapsed_time:.2f} seconds. Best internal {metric_name if models is not None else 'Metric'}: {best_model_metric if isinstance(best_model_metric, (int, float)) else best_model_metric}")
    return best_model_metric, elapsed_time


def run_lightautoml(train_df_orig, test_df_orig, target_col, roles, task: Task):
    """Запускает LightAutoML и возвращает метрику на тесте и время."""
    print(f"\n--- Running LightAutoML for task: {task.name} ---")
    automl = TabularAutoML(task=task, timeout=300, cpu_limit=4, memory_limit=16)
    score = np.nan
    elapsed_time = np.nan
    metric_name = 'Unknown'

    try:
        start_time = time.time()
        _ = automl.fit_predict(train_df_orig, roles=roles, verbose=1)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"LightAutoML fit_predict completed in {elapsed_time:.2f} seconds.")

        test_preds = automl.predict(test_df_orig).data
        print(f"Prediction on test data completed. Shape: {test_preds.shape}")

        y_test_true = test_df_orig[target_col]

        print("Calculating metric on test set...")
        if task.name == 'binary':
            metric_name = 'ROC AUC'

            if test_preds.ndim == 1 or test_preds.shape[1] == 1:
                test_preds_proba = test_preds.flatten()
            elif test_preds.shape[1] == 2:
                le_test_internal = LabelEncoder().fit(y_test_true)
                if 1 in le_test_internal.classes_:
                    pos_idx = list(le_test_internal.classes_).index(1)
                    test_preds_proba = test_preds[:, pos_idx]
                    print(f"Using column {pos_idx} for positive class probabilities.")
                else:
                    print("Warning: Class '1' not found. Using column 1 for probabilities.")
                    test_preds_proba = test_preds[:, 1]
            else:
                raise ValueError(f"Unexpected prediction array shape {test_preds.shape} for binary task.")

            if not pd.api.types.is_numeric_dtype(y_test_true):
                le = LabelEncoder()
                y_test_true_enc = le.fit_transform(y_test_true)
                print(f"Encoded target classes: {le.classes_}")
            else:
                y_test_true_enc = y_test_true

            score = roc_auc_score(y_test_true_enc, test_preds_proba)

        elif task.name == 'reg':
            metric_name = 'RMSE' # Имя метрики для вывода
            y_test_true_clean = y_test_true.fillna(y_test_true.median())
            score = mean_squared_error(y_test_true_clean, test_preds[:, 0], squared=False)
        else:
            raise ValueError("Unsupported task type for metric calculation")

        print("Metric calculation successful.")
        print(f"LightAutoML finished. Test {metric_name}: {score:.4f}. Time: {elapsed_time:.2f} seconds.")

    except Exception as e:
        print(f"LightAutoML failed during execution: {e}")
        import traceback
        traceback.print_exc()
        score = 'Error'
        if 'start_time' in locals() and np.isnan(elapsed_time):
            elapsed_time = time.time() - start_time
        elif np.isnan(elapsed_time):
            elapsed_time = 0
        print(f"LightAutoML failed after {elapsed_time:.2f} seconds.")

    return score, elapsed_time

def run_flaml(X_train, y_train, X_test, y_test, task_type, metric_name, time_budget=120):
    """Запускает FLAML и возвращает метрику на тесте и время."""
    print(f"\n--- Running FLAML for {task_type} ---")
    automl = AutoML()
    settings = {
        "time_budget": time_budget,
        "metric": metric_name,
        "task": task_type,
        "log_file_name": f"flaml_{task_type}.log",
        "n_jobs": -1,
        "seed": 42,
        "eval_method": "auto",
        "early_stop": True
    }
    score = np.nan
    elapsed_time = np.nan

    try:
        start_time = time.time()
        
        if task_type == 'classification' and not pd.api.types.is_numeric_dtype(y_train):
            le_flaml = LabelEncoder()
            y_train_enc = le_flaml.fit_transform(y_train)
            y_test_enc = le_flaml.transform(y_test)
        else:
            y_train_enc = y_train
            y_test_enc = y_test

        if not isinstance(X_train, pd.DataFrame):
             X_train_df = pd.DataFrame(X_train)
             X_test_df = pd.DataFrame(X_test)
        else:
             X_train_df = X_train
             X_test_df = X_test
            
        X_train_df.columns = X_train_df.columns.astype(str)
        X_test_df.columns = X_test_df.columns.astype(str)

        automl.fit(X_train_df, y_train_enc, **settings)
        end_time = time.time()
        elapsed_time = end_time - start_time

        if task_type == 'classification':
            test_preds_proba = automl.predict_proba(X_test_df)[:, 1]
            score = roc_auc_score(y_test_enc, test_preds_proba)
            metric_display_name = 'ROC AUC'
        elif task_type == 'regression':
            y_test_clean = pd.Series(y_test_enc).fillna(pd.Series(y_train_enc).median()).values
            test_preds = automl.predict(X_test_df)
            score = mean_squared_error(y_test_clean, test_preds, squared=False)
            metric_display_name = 'RMSE'
        else:
            raise ValueError("task_type must be 'classification' or 'regression'")

        print(f"FLAML finished in {elapsed_time:.2f} seconds. Best model: {automl.best_estimator}. Test {metric_display_name}: {score:.4f}")

    except Exception as e:
        print(f"FLAML failed: {e}")
        score = 'Error'
        elapsed_time = time.time() - start_time
        print(f"FLAML failed after {elapsed_time:.2f} seconds.")


    return score, elapsed_time

print("AutoML runner functions defined.")

AutoML runner functions defined.


In [5]:
results = {'flat_price': {}, 'user_churn': {}}

print("\n=== Starting User Churn Task (Classification) ===")
churn_train_path = 'train_churn.csv' 
churn_test_path = 'test_churn.csv'
churn_target = 'Churn'

X_train_c, y_train_c, X_test_c, y_test_c = preprocess_churn(churn_train_path, churn_test_path, churn_target)

train_df_c_orig = pd.read_csv(churn_train_path)
test_df_c_orig = pd.read_csv(churn_test_path)
train_df_c_orig.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df_c_orig.columns]
test_df_c_orig.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_df_c_orig.columns]
churn_target_clean = "".join (c if c.isalnum() else "_" for c in str(churn_target))

score_lp_c, time_lp_c = run_lazypredict(X_train_c, y_train_c, X_test_c, y_test_c, 'classification')
results['user_churn']['lazypredict'] = {'Metric': score_lp_c, 'Time': time_lp_c}

churn_roles = {'target': churn_target_clean}
churn_task = Task('binary')
score_lama_c, time_lama_c = run_lightautoml(train_df_c_orig, test_df_c_orig, churn_target_clean, churn_roles, churn_task)
results['user_churn']['LightAutoML'] = {'Metric': score_lama_c, 'Time': time_lama_c}

score_flaml_c, time_flaml_c = run_flaml(X_train_c, y_train_c, X_test_c, y_test_c, 'classification', 'roc_auc', time_budget=120) # Увеличим бюджет
results['user_churn']['FLAML'] = {'Metric': score_flaml_c, 'Time': time_flaml_c}

print("\n=== User Churn Task Finished ===")


=== Starting User Churn Task (Classification) ===
Preprocessing Churn data: train_churn.csv, test_churn.csv...
Churn preprocessing finished.

--- Running LazyPredict for classification ---


  7%|███                                         | 2/29 [00:00<00:05,  5.39it/s]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.7730138713745272, 'Balanced Accuracy': 0.6610627177700349, 'ROC AUC': 0.6610627177700349, 'F1 Score': 0.7607812536178268, 'roc_auc_score': 0.6610627177700349, 'Time taken': 0.23195409774780273}
{'Model': 'BaggingClassifier', 'Accuracy': 0.7843631778058008, 'Balanced Accuracy': 0.6734818317570931, 'ROC AUC': 0.6734818317570932, 'F1 Score': 0.771937712151358, 'roc_auc_score': 0.6734818317570932, 'Time taken': 0.15196657180786133}
{'Model': 'BernoulliNB', 'Accuracy': 0.755359394703657, 'Balanced Accuracy': 0.645980587356894, 'ROC AUC': 0.645980587356894, 'F1 Score': 0.7450657736064885, 'roc_auc_score': 0.645980587356894, 'Time taken': 0.023845672607421875}


 17%|███████▌                                    | 5/29 [00:00<00:01, 12.91it/s]

{'Model': 'CalibratedClassifierCV', 'Accuracy': 0.7755359394703657, 'Balanced Accuracy': 0.6659407665505227, 'ROC AUC': 0.6659407665505226, 'F1 Score': 0.763986576721334, 'roc_auc_score': 0.6659407665505226, 'Time taken': 0.07070541381835938}
{'Model': 'DecisionTreeClassifier', 'Accuracy': 0.7288776796973518, 'Balanced Accuracy': 0.6583084453293513, 'ROC AUC': 0.6583084453293513, 'F1 Score': 0.7319107550544387, 'roc_auc_score': 0.6583084453293513, 'Time taken': 0.038869619369506836}
{'Model': 'DummyClassifier', 'Accuracy': 0.7414880201765448, 'Balanced Accuracy': 0.5, 'ROC AUC': 0.5, 'F1 Score': 0.6314191974856022, 'roc_auc_score': 0.5, 'Time taken': 0.016569137573242188}
{'Model': 'ExtraTreeClassifier', 'Accuracy': 0.7187894073139974, 'Balanced Accuracy': 0.6229094076655052, 'ROC AUC': 0.6229094076655052, 'F1 Score': 0.7157225197634501, 'roc_auc_score': 0.6229094076655052, 'Time taken': 0.01956462860107422}


...

[20:17:35] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM completed
[20:17:35] Start fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ...


[20:17:35] Fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM finished. score = 0.816558786825847
[20:17:35] Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM fitting and predicting completed
[20:17:35] Start fitting Lvl_0_Pipe_1_Mod_2_CatBoost ...
[20:17:36] Fitting Lvl_0_Pipe_1_Mod_2_CatBoost finished. score = 0.8246747589359614
[20:17:36] Lvl_0_Pipe_1_Mod_2_CatBoost fitting and predicting completed
[20:17:36] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ... Time budget is 196.57 secs


Optimization Progress: 100%|█| 101/101 [00:17<00:00,  5.62it/s, best_trial=94, b

[20:17:54] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost completed
[20:17:54] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...


[20:17:55] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = 0.8246842400621142
[20:17:55] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[20:17:55] Time left 245.43 secs

[20:17:55] Layer 1 training completed.

[20:17:55] Blending: optimization starts with equal weights. Score = 0.8364838
[20:17:55] Blending: iteration 0: score = 0.8510616, weights = [0.76711875 0.         0.16106412 0.07181717 0.        ]
[20:17:55] Blending: iteration 1: score = 0.8511275, weights = [0.8123261  0.         0.11660836 0.07106554 0.        ]
[20:17:55] Blending: iteration 2: score = 0.8511329, weights = [0.82596105 0.         0.09596662 0.07807232 0.        ]
[20:17:55] Blending: no improvements for score. Terminated.

[20:17:55] Blending: best score = 0.8511329, best weights = [0.82596105 0.         0.09596662 0.07807232 0.        ]
[20:17:55] Automl preset training completed in 54.83 seconds

[20:17:55] Model description:
Final prediction for new objects (level 0) = 
	 0

In [6]:
print("\n=== Starting Flat Price Task (Regression) ===")
price_train_path = 'train_price.csv'
price_test_path = 'test_price.csv'
price_target = 'price'

X_train_p, y_train_p, X_test_p, y_test_p = preprocess_price(price_train_path, price_test_path, price_target)

train_df_p_orig = pd.read_csv(price_train_path)
test_df_p_orig = pd.read_csv(price_test_path)
train_df_p_orig.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df_p_orig.columns]
test_df_p_orig.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_df_p_orig.columns]
price_target_clean = "".join (c if c.isalnum() else "_" for c in str(price_target))

score_lp_p, time_lp_p = run_lazypredict(X_train_p, y_train_p, X_test_p, y_test_p, 'regression')
results['flat_price']['lazypredict'] = {'Metric': score_lp_p, 'Time': time_lp_p}

price_roles = {'target': price_target_clean}
price_task = Task('reg', metric='mse') # Используем 'mse' для инициализации
score_lama_p, time_lama_p = run_lightautoml(train_df_p_orig, test_df_p_orig, price_target_clean, price_roles, price_task)
results['flat_price']['LightAutoML'] = {'Metric': score_lama_p, 'Time': time_lama_p}

score_flaml_p, time_flaml_p = run_flaml(X_train_p, y_train_p, X_test_p, y_test_p, 'regression', 'rmse', time_budget=120) # Увеличим бюджет
results['flat_price']['FLAML'] = {'Metric': score_flaml_p, 'Time': time_flaml_p}

print("\n=== Flat Price Task Finished ===")


=== Starting Flat Price Task (Regression) ===
Preprocessing Price data: train_price.csv, test_price.csv...
Price preprocessing finished.

--- Running LazyPredict for regression ---


  2%|█                                           | 1/42 [00:00<00:29,  1.38it/s]

{'Model': 'AdaBoostRegressor', 'R-Squared': 0.39198463581711596, 'Adjusted R-Squared': 0.39151241707204987, 'RMSE': 23026676.384219553, 'Time taken': 0.7232785224914551, '<lambda>': 23026676.384219553}


  5%|██                                          | 2/42 [00:02<01:00,  1.51s/it]

{'Model': 'BaggingRegressor', 'R-Squared': 0.6401551341553663, 'Adjusted R-Squared': 0.6398756585023193, 'RMSE': 17714611.64975341, 'Time taken': 2.0618667602539062, '<lambda>': 17714611.64975341}
{'Model': 'BayesianRidge', 'R-Squared': 0.5079262241455897, 'Adjusted R-Squared': 0.5075440521239565, 'RMSE': 20715198.105511684, 'Time taken': 0.0720055103302002, '<lambda>': 20715198.105511684}


...

[20:26:34] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM completed
[20:26:34] Start fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ...


[20:26:38] Fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM finished. score = -763833348839860.2
[20:26:38] Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM fitting and predicting completed
[20:26:38] Start fitting Lvl_0_Pipe_1_Mod_2_CatBoost ...
[20:26:42] Fitting Lvl_0_Pipe_1_Mod_2_CatBoost finished. score = -968330622198421.4
[20:26:42] Lvl_0_Pipe_1_Mod_2_CatBoost fitting and predicting completed
[20:26:42] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ... Time budget is 155.78 secs


Optimization Progress:  53%|▌| 54/101 [02:38<02:17,  2.93s/it, best_trial=24, be

[20:29:20] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost completed
[20:29:20] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...


[20:29:30] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = -901757478303153.8
[20:29:30] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[20:29:30] Time left 57.74 secs

[20:29:30] Layer 1 training completed.

[20:29:30] Blending: optimization starts with equal weights. Score = -851591981656305.5000000
[20:29:30] Blending: iteration 0: score = -763833348839860.2500000, weights = [0. 0. 1. 0. 0.]
[20:29:30] Blending: no improvements for score. Terminated.

[20:29:30] Blending: best score = -763833348839860.2500000, best weights = [0. 0. 1. 0. 0.]
[20:29:30] Automl preset training completed in 242.40 seconds

[20:29:30] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) 

LightAutoML fit_predict completed in 242.40 seconds.
Prediction on test data completed. Shape: (9021, 1)
Calculating metric on test set...
Metric calculation successful.
LightAutoML finished. Test RMSE: 1536609

In [7]:
flat_price_results_list = []
for framework, data in results['flat_price'].items():
    metric_val = data['Metric'] if isinstance(data['Metric'], (int, float)) else 'N/A' if data['Metric'] != 'Error' else 'Error'
    time_val = data['Time'] if isinstance(data['Time'], (int, float)) else 'N/A' if data['Time'] != 'Error' else 'Error'
    flat_price_results_list.append({'Framework': framework, 'RMSE': metric_val, 'Время выполнения, сек.': time_val})

df_price = pd.DataFrame(flat_price_results_list)
df_price = df_price.set_index('Framework')

if 'Время выполнения, сек.' in df_price.columns:
     df_price['Время выполнения, сек.'] = pd.to_numeric(df_price['Время выполнения, сек.'], errors='coerce').round(2)

print("\n\nПоказатели метрик, полученных на датасете flat_price:")
print(df_price.to_markdown(floatfmt=".4f"))

user_churn_results_list = []
for framework, data in results['user_churn'].items():
    metric_val = data['Metric'] if isinstance(data['Metric'], (int, float)) else 'N/A' if data['Metric'] != 'Error' else 'Error'
    time_val = data['Time'] if isinstance(data['Time'], (int, float)) else 'N/A' if data['Time'] != 'Error' else 'Error'
    user_churn_results_list.append({'Framework': framework, 'ROC AUC': metric_val, 'Время выполнения, сек.': time_val})

df_churn = pd.DataFrame(user_churn_results_list)
df_churn = df_churn.set_index('Framework')

if 'Время выполнения, сек.' in df_churn.columns:
     df_churn['Время выполнения, сек.'] = pd.to_numeric(df_churn['Время выполнения, сек.'], errors='coerce').round(2)


print("\n\nПоказатели метрик, полученных на датасете user_churn:")
print(df_churn.to_markdown(floatfmt=".4f"))



Показатели метрик, полученных на датасете flat_price:
| Framework   |          RMSE |   Время выполнения, сек. |
|:------------|--------------:|-------------------------:|
| lazypredict | 15913574.4071 |                 331.4800 |
| LightAutoML | 15366090.8499 |                 242.4000 |
| FLAML       | 14825147.9141 |                 125.6500 |


Показатели метрик, полученных на датасете user_churn:
| Framework   |   ROC AUC |   Время выполнения, сек. |
|:------------|----------:|-------------------------:|
| lazypredict |    0.6950 |                   6.3500 |
| LightAutoML |    0.8193 |                  54.8400 |
| FLAML       |    0.8204 |                 120.6600 |
